In [ ]:
import copy
# xmlrpc_server.ipynb

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
from xmlrpc.client import Binary
import datetime
import pandas as pd
import pickle


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

# Тест
def ping():
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(surname, name, patronym, birthdate):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    frame.reset_index()
    
    exists = False
    for index, row in frame.iterrows():
        
        if (str(row['Surname']) == surname) & (str(row['Name']) == name) & (str(row['Patronym']) == patronym) & (str(row['Birth']) == birthdate):
            exists = True
        
    sum_info = surname + ' ' + name + ' ' + patronym + ', ' + birthdate
    if exists:
        return sum_info + ": "+ "bad_boy"
    else:
        return sum_info + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Бинарная передача данных
def send_back_binary(bin_data):
    data = bin_data.data
    return Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width):
            if type(img_arr[i][j]) is list:
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
            else:
                img_arr[i][j] = 255 - img_arr[i][j]
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')


def binary_val(val, border):
    if val < border:
        return 0
    else:
        return 255


# Бинаризация изображения по задаваемому порогу 1-255
def binary_img(bin_data, border):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width):
            try:
                img_arr[i][j][0] = binary_val(img_arr[i][j][0], border)
                img_arr[i][j][1] = binary_val(img_arr[i][j][1], border)
                img_arr[i][j][2] = binary_val(img_arr[i][j][2], border)
            except Exception:
                img_arr[i][j] = binary_val(img_arr[i][j], border)
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(binary_img, 'binary_img')


# Поворот относительно вертикали
def reverse(bin_data):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width // 2):
            temp = copy.deepcopy(img_arr[i][j])
            img_arr[i][j] = copy.deepcopy(img_arr[i][width - j - 1])
            img_arr[i][width - j - 1] = copy.deepcopy(temp)
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(reverse, 'reverse')


print("Listening on port 8008...")
server.serve_forever()


Listening on port 8008...


127.0.0.1 - - [16/Sep/2023 14:06:54] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2023 14:07:05] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2023 14:07:22] "POST /RPC2 HTTP/1.1" 200 -
